Author:
    
Abel Stanley
13517068

# Week 06 Handson - Text Classifier
In this week hands-on, we will create a sentiment analyzer on twitter using the concept of classification and text pre-processing that we have learned before. We will cover:<br>
a. text pre-processing,<br>
b. splitting data for training & testing and converting them into numerical features,<br>
c. training a classifier model and perform predictions on testing dataset,<br>
d. Evaluating performance of algorithm<br>

## Read dataset "tweets.csv"

In [61]:
import numpy as np
import pandas as pd

tweets = pd.read_csv('./tweets.csv', sep=",")# adjust with your own path
tweets.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1038,1,Sentiment140,that film is fantastic #brilliant
1,1804,1,Sentiment140,this music is really bad #myband
2,1693,0,Sentiment140,winter is terrible #thumbs-down
3,1477,0,Sentiment140,this game is awful #nightmare
4,45,1,Sentiment140,I love jam #loveit


## Milestone 01 (W01)
The given dataset is still a 'raw dataset' which includes some unwanted features, unwanted characters, etc.<br>
a. Select the `SentimentText` column as an attribute and the `Sentiment` column as a label (ground truth) for this study case<br><br>
b. In this M01.b, you have been provided a function template `pre_process` (see below) to perform all the pre-processing step to the all tweet data in the dataset. Complete pre-process function with all techniques that you have learned in the previous hands-on week (W03) for text pre-processing, so the all text attributes can be converted to `pre-processed text`, e.g., after being applied by: (i) tokenization, (ii) normalization, (iii) cleaning, (iv) stemming or lemmatization. Here, you will get `list of words`.<br><br>
c. Use the function that you have completed in M01.b, looped for each data row of `SentimentText` column. For each looping, you will get `list of words`. Append this `list of words` for each looping result in to list, so, will get `list of list`.<br><br>

d. Split (random & stratified) `list of list` you get in M01.c into `training data` and `testing data`. The testing dataset must be 20% from overall dataset. Print the total number of initial dataset, total number of training dataset and testing dataset. <br>


In [62]:
#put your code here for M01.a
X, y = tweets['SentimentText'], tweets['Sentiment']

print(X[0:2])
print(y[0:2])

0    that film is fantastic #brilliant
1     this music is really bad #myband
Name: SentimentText, dtype: object
0    1
1    1
Name: Sentiment, dtype: int64


In [63]:
#put your code here for M01.b
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

def tokenize(text):
    return word_tokenize(text)

def normalization(text):
    def to_lower_case(text):
        return [w.lower() for w in text]
    
    table = str.maketrans('', '', string.punctuation)
    def remove_punctuation(text):
        return [w.translate(table) for w in text]
    
    def remove_non_alphabetic(text):
        return [word for word in text if word.isalpha() ]
    
    stop_words = set(stopwords.words('english' ))
    def remove_stopwords(text):
        return [w for w in text if not w in stop_words ]
    
    return remove_stopwords(remove_non_alphabetic(remove_punctuation(to_lower_case(text))))

from nltk.stem import WordNetLemmatizer
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]
    

def pre_process(input_ori):
    '''
    Write code implementation for text pre-processing here. 
    Use what you have learned before about text pre-processing.
    
    Parameter:
    input_ori = raw data text (single tweet data)
    
    Return value:
    processed_tweet = 'list of words'
    
    
    '''
    processed_tweet = lemmatize(normalization(tokenize(input_ori)))
    # processed_tweet = ['dummy','dummy','dummy'] #this is only dummy data
    
    return processed_tweet

pre_process('that film is fantastic #brilliant')

['film', 'fantastic', 'brilliant']

In [64]:
#put your code here for M01.c
X_ = []
for text in X:
    X_.append(pre_process(text))
    
X_[0:2]

[['film', 'fantastic', 'brilliant'], ['music', 'really', 'bad', 'myband']]

In [65]:
#put your code here for M01.d
from sklearn.model_selection import StratifiedShuffleSplit

def splitData(data, label, train_sz, test_sz, seed=0):
    sss = StratifiedShuffleSplit(n_splits=1, train_size=train_sz, test_size=test_sz, random_state=seed)
    indices = sss.split(data, label)
    train_indices, test_indices = next(indices)
    
    train_splitted_data, train_splitted_label = data[train_indices], label[train_indices]
    test_splitted_data, test_splitted_label = data[test_indices], label[test_indices]
                                                                        
    return (train_splitted_data, train_splitted_label, test_splitted_data, test_splitted_label)

X_ = np.array(X_)
training_data = {} 
testing_data = {}
training_data['X'], training_data['y'], testing_data['X'], testing_data['y'] = splitData(X_, y, 0.8, 0.2)

## M02
a. Build `tfidf_model` by using codes below with `training data` you get in M01.d. (`TfidfVectorizer` is from scikit-learn)
```
def dummy(doc):
    return doc
tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy,
    preprocessor=dummy,
    token_pattern=None)
```
b. Transform `training data` and `testing data` you get in M01.d by using `tfidf_model` you get in M02.a. In this case, you will get numerical features, both from `training data` and `testing data`.<br><br>
c. Choose a classification algorithm (you may use library such as scikit-learn), and explain why you choose it.<br><br>
d. Train the classifier model, based on the algorithm you have chosen, by using numerical features of `training data` from M02.b.<br><br>
e. Make predictions on the numerical features of `testing dataset` you get in M02.b using the classifier model that you have trained.

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
# put your code here for M02.a
def dummy(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word', #''
    tokenizer=dummy ,
    preprocessor=dummy,
    token_pattern=None )
       
# train / learn from the given data
model = tfidf.fit(training_data['X'])

In [67]:
# put your code here for M02.b
training_data['X'] = model.transform(training_data['X'] ).toarray()
testing_data['X'] = model.transform(testing_data['X'] ).toarray()

Your explanation (M02.c):

Logistic regression is the appropriate regression analysis to conduct when the dependent variable is **dichotomous (binary)**.  Like all regression analyses, the logistic regression is a predictive analysis.  Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables.

Based on my research, we use logistic regression when:
1. We have a binary or dichotomous Y variable.
2. We have explanatory X-variables that we think are related to the Y-variable.
3. It is reasonable to think that the value the Y-variable takes on is like a coin flip where the probability of getting a 1 (“heads”) depends on the explanatory variables.

The prediction class in this particular dataset consists only of 2 types of classes, which means that the dependent variable is binary. The SentimentText column in the dataset serves as the X-variable which also correlates and explain the Y variable (Sentiment class). Thus, the dataset fits the preferred use-case of Logistic Regression.

In [68]:
y.value_counts()

0    974
1    958
Name: Sentiment, dtype: int64

In [69]:
# put your code here for M02.d
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# use Grid search to find the best parameters for Logistic Regression:
param_grid_lr = {
'solver': ['newton-cg', 'lbfgs', 'liblinear','sag','saga'],
'class_weight':[None, 'balanced'],
'C':np.logspace(-4, 4, 20),
'max_iter':[100, 250, 500]
}

gs_cv = GridSearchCV(LogisticRegression(random_state=0), param_grid_lr).fit(training_data['X'] , training_data['y'])
print("Best Score: ", gs_cv.best_score_)
print("Best Params: ", gs_cv.best_params_)

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceW

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceW

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceW

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver optio

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the c

...

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceW

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number 

C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceW

Best Score:  0.9883495145631068
Best Params:  {'C': 0.00026366508987303583, 'class_weight': 'balanced', 'max_iter': 100, 'solver': 'sag'}


C:\Users\Dell Warna Biru\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [70]:
# Train Logistic Regression with best parameters:
clf = LogisticRegression(random_state=0, C=0.00026366508987303583, class_weight='balanced', max_iter=100, solver='sag').fit(training_data['X'] , training_data['y'])

In [71]:
# put your code here for M02.e
from sklearn.metrics import classification_report

predictions = clf.predict(testing_data['X'])

## M03
After train the classification model and make prediction using that model, now you will evaluate the performance of your model against testing dataset.<br>
a. Calculate and print the accuracy of your model's predictions in M02.e against testing dataset ground truth<br>
b. What you can infer based on the result?<br>

In [72]:
#put your code here for Q03.a
print(classification_report(y_true = testing_data['y'], y_pred = predictions))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       195
           1       0.96      0.98      0.97       192

    accuracy                           0.97       387
   macro avg       0.97      0.97      0.97       387
weighted avg       0.97      0.97      0.97       387



Your answer (M03.b) :

Using logistic regression classifier, we get an Accuracy of 0.97 and f-1 score of 0.97, which is pretty good.

*definitions:*

1. Precision — TP/(TP+FP), meaning the proportion of points that model classify as positives are actually positives.
2. Recall — TP/(TP+FN), meaning the proportion of actual positives that are correctly classified by the model.
3. F1 score —the harmonic mean of precision and recall.
4. The support is the number of occurence of the given class in your dataset (so we have 195 of class 0 and 192 of class 1, which is a really well balanced dataset.

Sometimes accuracy might not be the most appropriate evaluation metric when class distribution is imbalanced. However in this case, class 0 and 1 is evenly distributed (974 vs 958), so Accuracy is a valid metric to measure model's performance.

Therefore, a Logistic Regression model is a suitable classifier model for this specific sentiment analysis case.